In [34]:
from BlackJack_classes import *

card_values = ['2','3','4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
card_suits = ['Hearts', 'Diamonds', 'Spades', 'Clubs']

In [15]:
class RL_Player(Player):
    """A reinforcement learning agent that uses a policy to determine moves"""
    
    def __init__(self):
        super().__init__()
        self.Q = np.zeros((17, 13, 2)) # action value function
        self.pi = np.random.rand(17,13,2) # policy
        
    def hitOrStay(self, faceup)

In [41]:
a_deck_of_cards = [Card(value, suit) for value in card_values for suit in card_suits]

aDeck = Deck(a_deck_of_cards)

aPlayer = HumanPlayer()
aDealer = Dealer()



for i in range(1,11):
    aGame = Game(i,aDeck, [aPlayer], aDealer)
    aGame.play()


Please enter your name ...Grant
Your cards are:  [10 of Hearts, 7 of Clubs] 
Your score is:  17 

Would you like to hit or stay? (enter "hit" or "stay")hit
Grant :  0 27 Dealer :  0 23
Your cards are:  [9 of Diamonds, 2 of Diamonds] 
Your score is:  11 

Would you like to hit or stay? (enter "hit" or "stay")hit
Your cards are:  [9 of Diamonds, 2 of Diamonds, 6 of Clubs] 
Your score is:  17 

Would you like to hit or stay? (enter "hit" or "stay")hit
Grant :  0 23 Dealer :  0 23
Your cards are:  [6 of Clubs, J of Diamonds] 
Your score is:  16 

Would you like to hit or stay? (enter "hit" or "stay")hit
Grant :  -1 25 Dealer :  1 18
Your cards are:  [A of Hearts, 7 of Spades] 
Your score is:  18 

Would you like to hit or stay? (enter "hit" or "stay")hit
Your cards are:  [A of Hearts, 7 of Spades, 2 of Hearts] 
Your score is:  20 

Would you like to hit or stay? (enter "hit" or "stay")hit
Your cards are:  [A of Hearts, 7 of Spades, 2 of Hearts, 7 of Clubs] 
Your score is:  17 

Would you l

In [19]:
# 
Q_sa = {}
for c in card_values:
    Q = np.zeros(16, 13 2)
    Q_sa[c] = Q
    


In [40]:
# first index is player score - ranges from 5 to 21 (17): to update value, correct index is gotten using player.score - 5
# second index is face up value of dealer - any card value (13): correct index is gotten card_value.index(card value)
# third index is actions - hit or stay (2) - arbitrarily assign hit to index 0, stay to index 1
Q = np.zeros((17, 13, 2))
pi = np.random.rand(17,13,2)

AttributeError: 'numpy.ndarray' object has no attribute 'normalise'

In [36]:
HumanPlayer()


16

In [136]:
all_aces = [Card('A', 'Spades'), Card('10', 'Hearts'), Card('3', 'Spades'), Card('3', 'Hearts')]
aplayer = Player()
aplayer.getHand(all_aces)
aplayer.calcScore()
print(aplayer.score, aplayer.bust)

17 False


In [88]:
print(aplayer.hand)

None


In [39]:
for i in range(0, 26):
    print(i)
    k = aDeck.deal(2)
    print(aDeck[0:3], k)

0
[5 of Spades, 5 of Diamonds, 8 of Hearts] [5 of Hearts, 5 of Clubs]
1
[8 of Hearts, 8 of Clubs, 8 of Spades] [5 of Spades, 5 of Diamonds]
2
[8 of Spades, 8 of Diamonds, 3 of Hearts] [8 of Hearts, 8 of Clubs]
3
[3 of Hearts, 3 of Clubs, 3 of Spades] [8 of Spades, 8 of Diamonds]
4
[3 of Spades, 3 of Diamonds, Q of Hearts] [3 of Hearts, 3 of Clubs]
5
[Q of Hearts, Q of Clubs, Q of Spades] [3 of Spades, 3 of Diamonds]
6
[Q of Spades, Q of Diamonds, 6 of Hearts] [Q of Hearts, Q of Clubs]
7
[6 of Hearts, 6 of Clubs, 6 of Spades] [Q of Spades, Q of Diamonds]
8
[6 of Spades, 6 of Diamonds, 7 of Hearts] [6 of Hearts, 6 of Clubs]
9
[7 of Hearts, 7 of Clubs, 7 of Spades] [6 of Spades, 6 of Diamonds]
10
[7 of Spades, 7 of Diamonds, 9 of Hearts] [7 of Hearts, 7 of Clubs]
11
[9 of Hearts, 9 of Clubs, 9 of Spades] [7 of Spades, 7 of Diamonds]
12
[9 of Spades, 9 of Diamonds, A of Hearts] [9 of Hearts, 9 of Clubs]
13
[A of Hearts, A of Clubs, A of Spades] [9 of Spades, 9 of Diamonds]
14
[A of Spades,

In [37]:
aDeck.reclaimCards()

In [38]:
aDeck[0:3]

[5 of Hearts, 5 of Clubs, 5 of Spades]